# Build pipeline with command function

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using command() function
- Create `Pipeline` using component defined by command() function

**Motivations** - This notebook explains how to define `CommandComponent` via command function, then use command component to build pipeline. The command component is a fundamental construct of Azure Machine Learning pipeline. It can be used to run a task on a specified compute (either local or on the cloud). The command component accepts `Environment` to setup required infrastructure. You can define a `Command` to run on this infrastructure with `inputs`. You can reuse the same `Component` with different pipeline.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment, BuildContext, JobService

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: C:\Users\ziqiwang\OneDrive - Microsoft\Documents\GitHub\azureml-examples\config.json


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9/resourceGroups/ZiqiTest/providers/Microsoft.MachineLearningServices/workspaces/ZiqiPipelineTest/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'c:\\Users\\ziqiwang\\OneDrive - Microsoft\\Documents\\GitHub\\azureml-examples\\sdk\\python\\jobs\\pipelines\\1d_pipeline_with_non_python_components', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x0000025FA2EF1A60>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 10, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x0000025FA2EF1E80>, '

# 2. Define Command object via command function 
Use `command` function to create a `Command` object which can be used in `@pipeline` function.


In [4]:
download_url = "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"
file_name = "data.csv"
custom_path = "azureml://datastores/workspaceblobstore/paths/custom_path/${{name}}/"
environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
# 1. Create a command component to download a input data
download_data = command(
    name="download-input",
    display_name="Download Data",
    description="Download a input from remote URL and return it in output.",
    tags=dict(),
    command="curl ${{inputs.url}} > ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(url=download_url, file_name=file_name),
    # example how to change path of output on step level,
    # please note if the output is promoted to pipeline level you need to change path in pipeline job level
    outputs=dict(
        output_folder=Output(type="uri_folder", mode="rw_mount", path=custom_path)
    ),
)
# 2. Create a R script command to train data
train_data_with_r = command(
    name="train-data-with-R",
    display_name="Train Data with R",
    description="Train data with R.",
    tags=dict(),
    command="Rscript train.R --data_folder ${{inputs.iris}}/${{inputs.file_name}}",
    environment=Environment(build=BuildContext(path="docker_context")),
    code="./src",
    inputs=dict(
        iris=Input(type="uri_folder"),
        file_name=file_name,
    ),
    outputs={},
    services={
        "Jupyterlab endpoint": JobService(job_service_type="jupyter_lab"),
        "Vscode endpoint": JobService(job_service_type="vs_code"),
        # "My_ssh": JobService(
        # job_service_type = "ssh",
        # properties={
        #   "sshPublicKeys":"<add-public-key>"
        # }
        # ) # uncomment add SSH Public Key to access job container via SSH
    },
)
# 3. Create a command component to show inputs
show_data = command(
    name="show-data",
    display_name="Show Data",
    description="Show data in command line and return it in output.",
    tags=dict(),
    command="cat ${{inputs.input_folder}}/${{inputs.file_name}} && cp ${{inputs.input_folder}}/${{inputs.file_name}} ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(input_folder=Input(type="uri_folder"), file_name=file_name),
    outputs=dict(output_folder=Output(type="uri_folder", mode="rw_mount")),
)

Class JobService: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [5]:
print(download_data)

type: command
inputs:
  url: https://azuremlexamples.blob.core.windows.net/datasets/iris.csv
  file_name: data.csv
outputs:
  output_folder:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceblobstore/paths/custom_path/${{name}}/
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1
properties: {}
component:
  name: download-input
  display_name: Download Data
  description: Download a input from remote URL and return it in output.
  type: command
  inputs:
    url:
      type: string
      default: https://azuremlexamples.blob.core.windows.net/datasets/iris.csv
    file_name:
      type: string
      default: data.csv
  outputs:
    output_folder:
      type: uri_folder
      mode: rw_mount
  command: curl ${{inputs.url}} > ${{outputs.output_folder}}/${{inputs.file_name}}
  environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1
  tags: {}
  is_deterministic: true
environment_variables: {}



# 3. Basic pipeline job

## 3.1 Build pipeline

Build a pipeline with predefined commands.

In [9]:
@pipeline(
    tags={"owner": "sdkteam", "tag": "tagvalue"},
)
def pipeline_with_non_python_components(url, file_name):
    """The hello world pipeline job."""
    download_data_node = download_data(url=url, file_name=file_name)
    train_data_with_r(
        iris=download_data_node.outputs.output_folder, file_name=file_name
    )
    show_data_node = show_data(
        input_folder=download_data_node.outputs.output_folder, file_name=file_name
    )

    return {"output_file": show_data_node.outputs.output_folder}


pipeline_job = pipeline_with_non_python_components(download_url, file_name)

# example how to change path of output on pipeline level
pipeline_job.outputs.output_file = Output(
    type="uri_folder", mode="rw_mount", path=custom_path
)

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"

In [10]:
print(pipeline_job)

display_name: pipeline_with_non_python_components
description: The hello world pipeline job.
type: pipeline
inputs:
  url: https://azuremlexamples.blob.core.windows.net/datasets/iris.csv
  file_name: data.csv
outputs:
  output_file:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceblobstore/paths/custom_path/${{name}}/
jobs:
  download_data_node:
    type: command
    inputs:
      url:
        path: ${{parent.inputs.url}}
      file_name:
        path: ${{parent.inputs.file_name}}
    outputs:
      output_folder:
        mode: rw_mount
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/custom_path/${{name}}/
    environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1
    properties: {}
    component:
      name: download-input
      display_name: Download Data
      description: Download a input from remote URL and return it in output.
      type: command
      inputs:
        url:
          type: string
      

# 3.2 Submit pipeline job

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)